# 🏁 Modulo 10 – Esercizio finale corso

# 🏠 DataHouse: Analisi del Mercato Immobiliare (King County Dataset)

## 🎯 Obiettivo
Realizzare un progetto Python completo che analizzi il dataset immobiliare **"House Sales in King County (USA)"**, contenente informazioni su oltre 20.000 case.  
Il progetto deve produrre un **report JSON** e alcuni **grafici di sintesi** sull’andamento dei prezzi e delle caratteristiche delle abitazioni.

## 🧱 Struttura della repository

```bash
datahouse/
│
├── data/
│ └── Housing.csv # Dataset originale
│
├── src/
│ ├── main.py # Script principale
│ ├── models.py # Classi: Immobile, Annuncio, ecc.
│ ├── utils_io.py # Lettura/scrittura file CSV/JSON
│ ├── analysis.py # Analisi e statistiche
│ ├── viz.py # Grafici e visualizzazioni
│ └── init.py
│
├── output/
│ ├── report.json # Report finale
│ ├── grafici/ # Cartella per grafici salvati
│ └── logs/ # File di log
│
├── README.md
├── pyproject.toml
└── .gitignore
```

---

## 📊 Dataset
Il file `Housing.csv` ha le seguenti colonne:

| Colonna | Descrizione |
|----------|-------------|
| id | ID univoco |
| date | Data vendita |
| price | Prezzo (USD) |
| bedrooms | Numero camere |
| bathrooms | Numero bagni |
| sqft_living | Superficie interna (piedi²) |
| sqft_lot | Superficie lotto esterno |
| floors | Numero piani |
| waterfront | 1 se vista mare, 0 altrimenti |
| view | Valutazione vista (0–4) |
| condition | Condizione dell’immobile (1–5) |
| grade | Qualità costruzione (1–13) |
| sqft_above | Superficie piani superiori |
| sqft_basement | Superficie seminterrato |
| yr_built | Anno costruzione |
| yr_renovated | Anno ristrutturazione |
| zipcode | CAP (zona) |
| lat, long | Coordinate |
| sqft_living15 | Superficie media abitazioni vicine |
| sqft_lot15 | Lotto medio abitazioni vicine |

---

## 🧩 Specifiche richieste

### 1️⃣ Analisi da implementare
- Prezzo medio per *zona (zipcode)*  
- Prezzo medio per *numero di camere*  
- Distribuzione del prezzo in funzione del *numero di bagni*  
- Prezzo medio per *vista mare (waterfront)*  
- Rapporto prezzo/superficie (`price_per_sqm`)  
- Case costruite dopo il 2000 (%)  
- Condizione media per classe di *grade*  
- Calcolo immobile più costoso ed economico  

### 2️⃣ Visualizzazioni
Usa **Matplotlib** o **Seaborn** per creare e salvare in `output/grafici/`:
1. 📊 Istogramma del prezzo (`price`)  
2. 📈 Prezzo medio per *numero camere*  
3. 🌊 Boxplot del prezzo per *vista mare (waterfront)*  
4. 🔥 Heatmap di correlazione delle variabili numeriche

### 3️⃣ Output finale
Il programma deve salvare in:
- `output/report.json` con statistiche aggregate
- `output/grafici/*.png` con i grafici
- `output/logs/app.log` con i log di esecuzione

---

## ⚙️ Requisiti tecnici

- Librerie: `pandas` (^2.2.0), `numpy` (^1.26.0), `matplotlib` (^3.8.0), `seaborn` (^0.13.0), `logging` (non serve installarla), `pathlib` (non serve installarla), `json` (non serve installarla)
- Struttura modulare con `import` corretti
- Type hints e docstring obbligatorie
- Log completo in `output/logs/app.log`

---

## 🕒 Tempo
- ⏱️ 3h sviluppo
- 🎤 1h Correzione + Domande

---

## 📊 Output atteso (esempio base)

```bash
=== DataHouse Start ===
📊 Dataset caricato: 21597 immobili
💡 Prezzo medio: 540.000 $
🏡 Prezzo medio per zona:
  98004 → 1.200.500 $
  98039 → 1.350.000 $
📈 Grafici salvati in output/grafici/
✅ Report: output/report.json
=== Fine ===



.

.

.

.

.

.

.

.

.

.

# ✅ Soluzione Proposta

## 🧰 `src/utils_io.py`

In [ ]:
import json
import logging
from pathlib import Path
import pandas as pd


def setup_logger():
    Path("output/logs").mkdir(parents=True, exist_ok=True)
    logger = logging.getLogger("datahouse")
    if logger.handlers:
        return logger
    logger.setLevel(logging.INFO)
    fh = logging.FileHandler("output/logs/app.log", encoding="utf-8")
    fmt = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s")
    fh.setFormatter(fmt)
    logger.addHandler(fh)
    return logger


def load_data(path: str, logger) -> pd.DataFrame:
    try:
        df = pd.read_csv(path)
        logger.info(f"✅ Caricato dataset: {len(df)} righe")
        return df
    except FileNotFoundError:
        logger.error(f"❌ File non trovato: {path}")
        raise
    except Exception as e:
        logger.error(f"Errore lettura CSV: {e}")
        raise


def save_json(data: dict, path: str):
    Path(path).parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)


## 🧩 `src/models.py`

In [ ]:
from dataclasses import dataclass

@dataclass
class House:
    price: float
    bedrooms: int
    bathrooms: float
    sqft_living: float
    sqft_lot: float
    floors: float
    waterfront: int
    view: int
    condition: int
    grade: int
    yr_built: int
    yr_renovated: int
    zipcode: int

    @property
    def price_per_sqm(self):
        """Ritorna prezzo per mq (1 mq = 10.764 piedi²)."""
        if self.sqft_living == 0:
            return 0
        return self.price / (self.sqft_living / 10.764)

    def is_modern(self):
        """True se costruita o ristrutturata dopo il 2000."""
        return self.yr_renovated >= 2000 or self.yr_built >= 2000


## 📈 `src/analysis.py`

In [ ]:
import pandas as pd
import numpy as np

def add_derived(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["price_per_sqm"] = df["price"] / (df["sqft_living"] / 10.764)
    df["modern"] = (df["yr_built"] >= 2000) | (df["yr_renovated"] >= 2000)
    return df


def compute_stats(df: pd.DataFrame) -> dict:
    stats = {
        "count": len(df),
        "price_mean": round(df["price"].mean(), 2),
        "price_median": round(df["price"].median(), 2),
        "price_min": float(df["price"].min()),
        "price_max": float(df["price"].max()),
        "avg_price_per_sqm": round(df["price_per_sqm"].mean(), 2),
        "modern_pct": round(df["modern"].mean() * 100, 2),
    }
    stats["price_by_zip"] = df.groupby("zipcode")["price"].mean().round(2).head(10).to_dict()
    stats["price_by_bedrooms"] = df.groupby("bedrooms")["price"].mean().round(2).to_dict()
    stats["price_by_waterfront"] = df.groupby("waterfront")["price"].mean().round(2).to_dict()
    stats["cond_by_grade"] = df.groupby("grade")["condition"].mean().round(2).to_dict()
    return stats


## 🎨 `src/viz.py` 

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd


def plot_hist_price(df: pd.DataFrame):
    Path("output/grafici").mkdir(parents=True, exist_ok=True)
    plt.figure(figsize=(8, 5))
    sns.histplot(df["price"], bins=50, color="skyblue", kde=True)
    plt.title("Distribuzione Prezzi Case")
    plt.xlabel("Prezzo ($)")
    plt.tight_layout()
    plt.savefig("output/grafici/hist_price.png", dpi=150)
    plt.close()


def plot_price_by_bedrooms(df: pd.DataFrame):
    plt.figure(figsize=(8, 5))
    sns.barplot(x="bedrooms", y="price", data=df, estimator="mean", ci=None)
    plt.title("Prezzo medio per numero camere")
    plt.savefig("output/grafici/price_by_bedrooms.png", dpi=150)
    plt.close()


def plot_price_by_waterfront(df: pd.DataFrame):
    plt.figure(figsize=(6, 5))
    sns.boxplot(x="waterfront", y="price", data=df)
    plt.title("Prezzo per vista mare (0=no, 1=sì)")
    plt.savefig("output/grafici/price_by_waterfront.png", dpi=150)
    plt.close()


def plot_corr_heatmap(df: pd.DataFrame):
    corr = df[["price", "sqft_living", "bedrooms", "bathrooms", "grade", "condition"]].corr()
    plt.figure(figsize=(6, 4))
    sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Correlazione tra variabili numeriche")
    plt.tight_layout()
    plt.savefig("output/grafici/correlation_heatmap.png", dpi=150)
    plt.close()


## 🚀 `src/main.py` 

In [ ]:
from utils_io import setup_logger, load_data, save_json
from analysis import add_derived, compute_stats
from viz import (
    plot_hist_price,
    plot_price_by_bedrooms,
    plot_price_by_waterfront,
    plot_corr_heatmap,
)


def main():
    logger = setup_logger()
    logger.info("=== DataHouse avviato ===")

    df = load_data("data/kc_house_data.csv", logger)

    df = add_derived(df)
    stats = compute_stats(df)
    save_json(stats, "output/report.json")

    logger.info("Report generato in output/report.json")

    plot_hist_price(df)
    plot_price_by_bedrooms(df)
    plot_price_by_waterfront(df)
    plot_corr_heatmap(df)

    logger.info("Grafici salvati in output/grafici/")
    logger.info("=== Esecuzione completata ===")


if __name__ == "__main__":
    main()


## 📦 `pyproject.toml` 

In [ ]:
[tool.poetry]
name = "datahouse"
version = "0.1.0"
description = "Analisi del mercato immobiliare basata sul dataset King County House Sales"
authors = ["Nome Cognome <email@example.com>"]
readme = "README.md"
license = "MIT"
packages = [{ include = "src" }]

[tool.poetry.dependencies]
python = ">=3.10,<3.13"
pandas = "^2.2.0"
numpy = "^1.26.0"
matplotlib = "^3.8.0"
seaborn = "^0.13.0"
requests = "^2.31.0"

[tool.poetry.group.dev.dependencies]
black = "^24.0.0"
flake8 = "^7.0.0"

[build-system]
requires = ["poetry-core"]
build-backend = "poetry.core.masonry.api"


## 🧾 `README.md` 

# 🏠 DataHouse – Analisi del Mercato Immobiliare (King County)

Progetto finale del corso **Python Base**  
Durata: 3 ore di sviluppo + 1 ora di presentazione  
Autori: *[Nome Cognome 1]*, *[Nome Cognome 2]*

---

## 🎯 Obiettivo
Analizzare i dati immobiliari di King County (Seattle, USA)  
per estrarre statistiche, generare visualizzazioni e creare un report JSON riassuntivo.

---

## 📁 Struttura della repo

```bash
datahouse/
│
├── data/
│ └── Housing.csv # Dataset originale
│
├── src/
│ ├── main.py # Script principale
│ ├── models.py # Classi: Immobile, Annuncio, ecc.
│ ├── utils_io.py # Lettura/scrittura file CSV/JSON
│ ├── analysis.py # Analisi e statistiche
│ ├── viz.py # Grafici e visualizzazioni
│ └── init.py
│
├── output/
│ ├── report.json # Report finale
│ ├── grafici/ # Cartella per grafici salvati
│ └── logs/ # File di log
│
├── README.md
├── pyproject.toml
└── .gitignore
```


---

## ⚙️ Setup ambiente

1️⃣ Inizializza progetto poetry:
```bash
poetry init

This command will guide you through creating your pyproject.toml config.

Package name [datahouse]:  
Version [0.1.0]:  
Description []:  Esercizio finale riepilogativo
Author [None, n to skip]:  Nome Cognome
License []:  
Compatible Python versions [>=3.12]:  

Would you like to define your main dependencies interactively? (yes/no) [yes] 
        You can specify a package in the following forms:
          - A single name (requests): this will search for matches on PyPI
          - A name and a constraint (requests@^2.23.0)
          - A git url (git+https://github.com/python-poetry/poetry.git)
          - A git url with a revision         (git+https://github.com/python-poetry/poetry.git#develop)
          - A file path (../my-package/my-package.whl)
          - A directory (../my-package/)
          - A url (https://example.com/packages/my-package-0.1.0.tar.gz)
        
Package to add or search for (leave blank to skip): pandas@^2.2.0       
Adding pandas@^2.2.0

Add a package (leave blank to skip): numpy@^1.26.0
Adding numpy@^1.26.0

Add a package (leave blank to skip): matplotlib@^3.8.0
Adding matplotlib@^3.8.0

Add a package (leave blank to skip): seaborn@0.13.0
Adding seaborn@0.13.0

Add a package (leave blank to skip): requests@^2.31.0
Adding requests@^2.31.0

Add a package (leave blank to skip): 

Would you like to define your development dependencies interactively? (yes/no) [yes] no
Generated file
```

2️⃣ Installa le dipendenze:
```bash
poetry install
poetry lock
```
3️⃣ Entra nell’ambiente virtuale:
```bash
poetry env activate
```
se il kernel non dovesse essere disponibile seguire le istruzione del modulo 1 per leggerlo forzatamente.

## ▶️ Esecuzione

Lancia il programma principale:
```bash
python src/main.py
```

Output attesi:
- `output/report.json` → statistiche aggregate
- `output/grafici/*.png` → grafici Seaborn
- `output/logs/app.log` → log dettagliato

## 📊 Dataset

Usa il dataset **King County House Sales** (formato CSV),
contenente oltre 20.000 case con colonne come:
- `price`, `bedrooms`, `bathrooms`, `sqft_living`, `floors`, `view`, `condition`, `grade`, `zipcode`, `ecc`.

## 🧠 Analisi principali

- Prezzo medio per zona (zipcode)
- Prezzo medio per numero camere
- Distribuzione prezzo per vista mare
- Condizione media per grade
- Percentuale case moderne (post-2000)
- Heatmap correlazioni numeriche

## 🎨 Grafici prodotti

| File                      | Descrizione                |
| ------------------------- | -------------------------- |
| `hist_price.png`          | Distribuzione prezzi       |
| `price_by_bedrooms.png`   | Prezzo medio per camere    |
| `price_by_waterfront.png` | Prezzo per vista mare      |
| `correlation_heatmap.png` | Correlazioni tra variabili |



## ✅ Output finale

Eseguendo:

```bash
poetry run python src/main.py
```

**🖥️ In console:**

```bash
=== DataHouse avviato ===
✅ Caricato dataset: 21597 righe
Report generato in output/report.json
Grafici salvati in output/grafici/
=== Esecuzione completata ===
```

**📁 In `output/` trovi:**

- `report.json` con tutte le statistiche **aggregate**
- Grafici:
    - `hist_price.png`
    - `price_by_bedrooms.png`
    - `price_by_waterfront.png`
    - `correlation_heatmap.png`
- Log in `output/logs/app.log`